In [ ]:
pip install kaggle

In [ ]:
# Create the .kaggle directory in the home directory
!mkdir -p ~/.kaggle

# Check if kaggle.json exists in the current directory
!ls kaggle.json

# Copy the kaggle.json file to the .kaggle directory
!cp kaggle.json ~/.kaggle/

# Change permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

# Verify the file has been copied and permissions are set
!ls -l ~/.kaggle/kaggle.json


ls: cannot access 'kaggle.json': No such file or directory
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
ls: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#Api to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:00<00:00, 123MB/s]
100% 80.9M/80.9M [00:00<00:00, 112MB/s]


In [ ]:
#extracting the compressed dataset
#python built-in library to extrat zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
#open file and extract

with ZipFile(dataset , 'r') as zip:   #r for read
     zip.extractall()
     print("dataset is extracted")

FileNotFoundError: [Errno 2] No such file or directory: '/content/sentiment140.zip'

Importing dependencies


In [ ]:

import numpy as np
import pandas as pd  #to structure data
import re #regular expressing for pattern matching
from nltk.corpus import stopwords #naturallanguage and corpus is module
from nltk.stem.porter import PorterStemmer #reduce words into root words
from sklearn.feature_extraction.text import TfidfVectorizer  #library for converting textual data imto numerical data
from sklearn.model_selection import train_test_split #split original data into test amd training data
from sklearn.linear_model import LogisticRegression #training the data
from sklearn.metrics import accuracy_score #cALCULATE accuracy of machine learning model

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
#printing stopwords in english
print(stopwords.words('english'))  #that doesnt add any influential(importance) meaning to our text so we will remove to reduce the size of data it


Data Processing

In [ ]:
#loading data from csv file to pandas data frame
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1' )

In [ ]:
#for checking the number of rows and columns. row= total no of tweets
twitter_data.shape

In [ ]:




#printing the first 5 rows of the dataframe
twitter_data.head()

In [ ]:
#naming columns and reading dataset again
column_names = ['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding = 'ISO-8859-1' )

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting the number of missing values/text inthe dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#checking the distribution of target column if distribution is uneven we have to do upsampling and downsampling to make it even
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

convert the target(label) 4 to 1


In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Tweet

1 --> Positive Tweet

**`Stemming`**
Stemming is the process of reducing a word to its root word




In [ ]:
port_stem = PorterStemmer()

In [ ]:
def steming(content): #content is each tweet
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)   #remove all the things that are not alphabets
  stemmed_content = stemmed_content.lower()  #convert all letters to lowercase
  stemmed_content = stemmed_content.split()  #split all words form tweet andput them in  list
  #we are taking each stemmed word and checking if it is present in stopwords
  #stem will reduce word to its keyword,if particular word is present in stop words, stemming wouldnot be performed and that words would be ignored
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #join all words from particular tweet
  stemmed_content = ' ' .join(stemmed_content)
  return stemmed_content

Applying the above function(template) to the text(tweets)


In [ ]:
#making another column and applying steming

twitter_data['stemmed_content'] = twitter_data['text'].apply(steming)  #28 minutes to complete this execution

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#seperating the data(tweet) and the label(target 0,1)
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values



In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into test data and training data



In [ ]:
#train_test_split function is used to split the data into train data and test data
#X_train will contain all the training data tweets, Y_train contain the corresponding targets(X_train)
#X_test contain the test data tweets and Y_test  contain whether it is positiv or negative(label for X_test data)
#test size mean how many data points are needed for test data point. 0.2 means 20% of the data will be the test data and 80% means wil be training data
#stratify means equal distribution is needed of zero and ones . 50-50 percentage in training and test data both
#random state splitting will be replicated. if change it or doesnot mention it data will split differently


X_train , X_test , Y_train ,Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


converting the text data into numerical data by feature extraction method called vectorizer


In [ ]:
#ML model doesnot understand text data so converting it into numerical
#convert text into numerical data. it will consider each word and will check that how much time it is repeated
#it and get some importance value convert text to numerical value so each word will have that specific value
#so ml model will correspond that importance value to either 0 or 1. Each word will get an importance score based on how much time it is repeated and ml model will check if it is positive tweet or negative

vectorizer = TfidfVectorizer()
#Transforming(converting data to numerical values) & fitting(trying to anaylze the nature of data) test data and training data
X_train =  vectorizer.fit_transform(X_train)   #all training data will be taken and score will be given to individual words so we will trasforming the
X_test =   vectorizer.transform(X_test) #test data has to be transformed based on the traing data has been transformed

In [ ]:
print(X_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [ ]:
print(X_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

**Training the Machine learning model**




**Logistic Regression**

In [ ]:
#Logistic regression is classification model that is used to classify data points into different classes
#two classes positive tweet and negative tweet
model = LogisticRegression(max_iter=1000) #maximum  number of times the  model has to go through data

In [ ]:
model.fit(X_train, Y_train) #fit our data to model and it will map the positiv tweets to label 1 1 and will map the neg to 0.

LogisticRegression(max_iter=1000)

Model Evaluation


Accuracy score


In [ ]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train) #model has to find what is the target whether it it a positive or negative, predict values as 0 or  1
training_data_accuracy = accuracy_score(Y_train, X_train_prediction) #now will compare the values predicted by model with true labels(y_train values)

In [ ]:
print('Accuracy score on the training data: ', training_data_accuracy)

Accuracy score on the training data:  0.81018984375


In [ ]:
#accuracy score on the test data
X_test_prediction = model.predict(X_test) #model has to find what is the target whether it it a positive or negative, predict values as 0 or  1
test_data_accuracy = accuracy_score(Y_test, X_test_prediction) #now will compare the values predicted by model with true labels(y_train values)

In [ ]:
print('Accuracy score on the test data: ', test_data_accuracy)

Accuracy score on the test data:  0.7780375


saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename, 'wb'))  #binaryformat

using saved model for future predictions

In [ ]:
#loading te saved model
loaded_model = pickle.load(open('/content/trained_model.sav','rb' )) #trained model path^

In [ ]:
#test the data that model hasnot seen
X_new = X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

else:
    print('Positive tweet')

NameError: name 'X_test' is not defined